In [1]:
import math
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
from matplotlib import cm
import numpy as np
import numpy.ma as ma 
import napari
import os
import pandas as pd
import random
import subprocess
import time

from aicsimageio import AICSImage, imread 
from scipy.ndimage import center_of_mass
from scipy.ndimage import shift as img_shift
from scipy.signal import correlate2d
from scipy.signal import correlate2d
from math import isclose 
from scipy.stats.mstats import pearsonr
from PIL import Image, ImageDraw
from skimage.io import imsave
from skimage.filters import threshold_otsu, gaussian
from skimage.morphology import medial_axis, skeletonize,binary_closing, skeletonize_3d,dilation, erosion, remove_small_objects
from skimage.registration import phase_cross_correlation
from skimage.segmentation import flood_fill
from scipy import ndimage as ndi
from skimage import color
from sklearn import neighbors
from scipy.spatial import distance, KDTree
from skimage import data, util, filters, color
from skimage.segmentation import watershed, random_walker
import skimage.measure as measure
# import processing functions
from processing.processing_functions import *

from DouglasPeucker import DouglasPeucker



# import machine learning modules
import torch
from torchvision import transforms, utils
from torch.utils.data import DataLoader
from monai.networks.nets import UNet
from monai.inferers.inferer import SliceInferer



2023-01-24 11:01:20.918701: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 11:01:21.227624: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-24 11:01:21.227639: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-24 11:01:21.266133: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-24 11:01:21.953658: W tensorflow/stream_executor/platform/de

In [2]:
# Change output figure size
# ...needs to be in its own cell for some reason...

plt.rcParams['figure.figsize'] = [16, 10]

In [3]:
# Image for Segementaion
IMAGE = 'pyNeuroDCNN/00_Live6-7-2018_08-53-23.tif'

# Min Pixel Length for Seg
PIX = 10

In [4]:
def model_predict(image, mode, spatial_dim):
    path = os.getcwd()

    if spatial_dim == 3:
        # use AI assistance
        lateral_steps = 64
        axial_steps = 16
        patch_size = (axial_steps, lateral_steps, lateral_steps)
        batch_size = 64
        dim_order = (0,4,1,2,3)
        orig_shape = image.shape
        
        patch_transform = transforms.Compose([MinMaxScalerVectorized(),
                                patch_imgs(xy_step = lateral_steps, z_step = axial_steps, patch_size = patch_size, is_mask = False)])

        processed_test_img = MyImageDataset(raw_img = image,
                                            transform = patch_transform,
                                            img_order = dim_order)
        
        # img_dataloader = DataLoader(processed_test_img, batch_size = 1)

        reconstructed_img = inference(processed_test_img,f'{path}/models/{mode}.onnx', batch_size, patch_size, orig_shape)
        reconstructed_img = reconstructed_img.astype(int)

        # soma category is inferenced for only "Neuron" => change all the soma labels (1) to dendrite labels (2)
        if len(np.unique(reconstructed_img)) == 2:
            reconstructed_img[reconstructed_img==1] = 2
            return reconstructed_img, len(np.unique(reconstructed_img))+1
        else:
            return reconstructed_img, len(np.unique(reconstructed_img))

    if spatial_dim == 2:
        # currently lateral steps are fixed as 512 due to the model being trained on full slices of 512x512.
        model_path = f'{path}/models/2D_{mode}.pth'

        checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
        lateral_steps = 512
        patch_size = (lateral_steps, lateral_steps)
        batch_size = 1
        input_chnl = 1
        output_chnl = 4
        norm_type = "batch"
        dropout = 0.1

        model = UNet(spatial_dims=2, 
                    in_channels = input_chnl,
                    out_channels = output_chnl,
                    channels = (32, 64, 128, 256, 512),
                    strides=(2, 2, 2, 2),
                    num_res_units=2,
                    norm = norm_type,
                    dropout = dropout)

        model.load_state_dict(checkpoint['model_state_dict'])
        model = model.to("cpu")
        inferer = SliceInferer(roi_size=patch_size, sw_batch_size=batch_size, spatial_dim = 0, progress = True)

        raw_transform = transforms.Compose([MinMaxScalerVectorized()])
        processed_img_dataset = WholeVolumeDataset(raw_img=image,
                                           raw_transform=raw_transform)
        
        processed_img, _ = next(iter(processed_img_dataset))
        processed_img = torch.unsqueeze(processed_img, dim = 0)

        with torch.no_grad():
            output = inferer(inputs = processed_img, network=model)
            print(output.shape)
            probabilities = torch.softmax(output,1)
            print(probabilities.shape)
            pred = to_numpy(torch.argmax(probabilities, 1)).astype(np.int16)

        # soma category is inferenced for only "Neuron" => change all the soma labels (1) to dendrite labels (2)
        if len(np.unique(pred)) == 2:
            pred[pred==1] = 2
            return pred, len(np.unique(pred))+1
        else:
            return pred, len(np.unique(pred))

In [5]:
data1 = imread(IMAGE)
img_slice = data1[ 0, 0, :, :, :]
print(img_slice.shape)
labels, other = model_predict(img_slice,"Soma+Dendrite", 2)

(190, 512, 512)
reading in existing image


100%|█████████████████████████████████████████| 190/190 [00:31<00:00,  5.97it/s]


torch.Size([1, 4, 190, 512, 512])
torch.Size([1, 4, 190, 512, 512])


In [6]:
labels[labels==3]=0


In [7]:
results_viewer = napari.Viewer()
results_viewer.add_image(data1)
results_viewer.add_labels(labels)

<Labels layer 'labels' at 0x7fc566022dd0>

2023-01-24 11:03:13,084 - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


In [8]:
def returnpoints_3D(array):
    test_array =array.copy()
    window = np.zeros([3,3])
    points = []
    for z in range(test_array.shape[0]):
        for i in range(512-3):
            for j in range(512-3):
                if test_array[z,i+1,j+1]==1:
                    window=test_array[z,i:i+3, j:j+3]
                    if np.sum(window)<=3:
                        if False ==((window[0,0]==True and window[2,2]==True) or (window[0,2]==True and window[2,0]==True)):

                            if np.sum(window[1,:])<=2 and  np.sum(np.sum(window[:,1]))<=2:
                                points.append((z,i+1,j+1))
    return points
                                              
                                              
def returnpoints_2D(array):
    test_array =array.copy()
    window = np.zeros([3,3])
    points = []
    for i in range(512-3):
        for j in range(512-3):
            if test_array[i+1,j+1]==1:
                window=test_array[i:i+3, j:j+3]
                if np.sum(window)<=3:
                    if False ==((window[0,0]==True and window[2,2]==True) or (window[0,2]==True and window[2,0]==True)):

                        if np.sum(window[1,:])<=2 and  np.sum(np.sum(window[:,1]))<=2:
                            points.append((j+1,i+1))
    return points
                                                               

In [9]:
# Create an directory to store outputs:
if not os.path.exists('results/'):
    os.makedirs('results')

## Keys for the pixel classification results:
* 0 - Background
* 1 - Soma
* 2 - Dendrites
* 3 - Junk



In [10]:
"""# Threshold and mask the image
threshold_image = image.copy()
img_thresh = threshold_otsu(image)
threshold_image[threshold_image< img_thresh]= 0
threshold_image[threshold_image>0]= 1
gau_thresh_img = gaussian(threshold_image.copy())
gau_thresh_img2 = gau_thresh_img.copy()
gau_thresh_img2[gau_thresh_img2>(img_thresh*.1)]=1

# Inspect the neuron
test = napari.Viewer()
test.add_image(image, name='Image', scale=(5, 1, 1), colormap='gray', blending='additive')
test.add_image(gau_thresh_img2, name='Gau Image',  scale=(5, 1, 1), colormap='green', blending='additive')
test.add_image(gau_thresh_img, name='Threshold Image', scale=(5, 1, 1), colormap='gray', blending='additive')"""

"# Threshold and mask the image\nthreshold_image = image.copy()\nimg_thresh = threshold_otsu(image)\nthreshold_image[threshold_image< img_thresh]= 0\nthreshold_image[threshold_image>0]= 1\ngau_thresh_img = gaussian(threshold_image.copy())\ngau_thresh_img2 = gau_thresh_img.copy()\ngau_thresh_img2[gau_thresh_img2>(img_thresh*.1)]=1\n\n# Inspect the neuron\ntest = napari.Viewer()\ntest.add_image(image, name='Image', scale=(5, 1, 1), colormap='gray', blending='additive')\ntest.add_image(gau_thresh_img2, name='Gau Image',  scale=(5, 1, 1), colormap='green', blending='additive')\ntest.add_image(gau_thresh_img, name='Threshold Image', scale=(5, 1, 1), colormap='gray', blending='additive')"

In [11]:
image = data1

In [12]:
def seg_skel(image, img2skel, twoD=True):
    # Takes an skeletonized image and segments the skeleton per plan
    # Returns segs as unique values in 3D array
    
    # 3D per volume
    if twoD==False:
        print('Skeltonize in 3D')
        skel = skeletonize_3d(neuron_mask)
    
    # 2D per plane
    if twoD==True:
        img2skel = img2skel/np.max(img2skel)
        skel = np.zeros(img2skel.shape)
        for u in range(img2skel.shape[0]):
            skel[u,:,:]  = skeletonize(img2skel[u,:,:])
    segsperplane = np.zeros(image.shape)
    for i in range(skel.shape[0]):
        edges = filters.sobel(skel[i,:,:])
        plane = skel[i,:,:]
        seeds = np.zeros((512,512))
        foreground, background = 1, 2
        seeds[plane <.5] = background
        seeds[skel[i,:,:] > .5] = foreground
        ws = watershed(edges, seeds)
        segments = measure.label(ws == foreground)
        temp_max = np.max(segsperplane)
        segments = segments+temp_max
        segments[segments ==temp_max]=0
        segsperplane[i,:,:] = segments
    return segsperplane

image = data1[0,0,:,:,:]
print(labels.shape)
dendrites = np.zeros_like(labels[0,:,:,:])
dendrites[labels[0,:,:,:]==2]=1
dendrites = np.array(dendrites, bool)
dendrites = remove_small_objects(dendrites, 500, connectivity=10)
test = napari.Viewer()
test.add_image(dendrites, name='Neuron', scale=(5, 1, 1), colormap='magenta', blending='additive')

# Select feature to segment
neuron_mask = dendrites
# Run segmentation and skeletonization
segsperplane = seg_skel(image, dendrites, False)
segsperplane = segsperplane.astype(int)
# Inspect results
test = napari.Viewer()
test.add_image(neuron_mask, name='Neuron', scale=(5, 1, 1), colormap='magenta', blending='additive')
test.add_labels(segsperplane, name='Segments', scale=(5, 1, 1), blending='additive')

(1, 190, 512, 512)
Skeltonize in 3D


<Labels layer 'Segments' at 0x7fc57cc12dd0>

In [13]:
def locationMinus(A, B):
    return (A[0] - B[0], A[1] - B[1])

def normDelta(p1, p2):
    x, y = locationMinus(p1, p2)
    sz = math.sqrt(x*x + y*y)
    if sz ==0:
        sz=1
    return (x/sz, y/sz,)

# Given a selected point and an additional neightboring point
# returns a point perpendicular to the line joining these points
def caliperPoints(origin, point, ifNext):
    x, y =  normDelta(point, origin)
    if ifNext:
        vector = np.array([y, -x])
    else:
        vector = np.array([-y, x])
    return vector

def return_corners(firstP, secP):

    pC = caliperPoints(firstP, secP, ifNext=False)
    pCprime = pC * 3
    radiiPoint = pCprime+firstP
    negRadiiPoint = -pCprime +firstP

    negX = int(negRadiiPoint[0])
    posX = int(radiiPoint[0])
    posY = int(negRadiiPoint[1])
    negY = int(radiiPoint[1])
    

    
    return(negX, posY, posX, negY)

def segmentGen(t1, t2, plane):


    x1, y1, x2, y2 = return_corners(t1, t2)
    x3, y3, x4, y4 = return_corners(t2, t1)


    polygon = [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]

    img = Image.new('L', (512, 512), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    block = np.array(img)
    binary_image = plane.copy()
    binary_image[binary_image>0]=1
    segment = block*binary_image
    return block, segment

def return_smaller_seg(p1, p2, threshold, midpoints):
    if round(math.hypot(p1[1] - p2[1], p1[0] - p2[0])) > threshold:
        mp = (round((p1[0] + p2[0])/2), round((p1[1] + p2[1])/2))
        fp = p1
        subpoint1 = return_smaller_seg(fp, mp, threshold, midpoints)
        if subpoint1 is False:
            midpoints.append(mp)
        sp = p2
        subpoint2 = return_smaller_seg(mp, sp, threshold, midpoints)
         
        if subpoint2 is False:
            midpoints.append(mp)
        return midpoints
    else:
        return False

In [14]:
def find_skeleton_points(points, skelly_image):
    skelly_image[skelly_image>0]=1
    end_points = []
    y_points = []

    for point in range(points[0].shape[0]):
        i = points[0][point]
        j = points[1][point]
        window=skelly_image[i-1:i+2, j-1:j+2]
        '''        if False ==((window[0,0]==True and window[2,2]==True) or (window[0,2]==True and window[2,0]==True)):
            if np.sum(window[1,:])<=2 and  np.sum(np.sum(window[:,1]))<=2:
                end_points.append((j,i))'''
        if np.sum(window)<=2:
            end_points.append((j,i))
        if np.sum(window)==4:
            y_points.append((j,i))
    #end_points = np.array(end_points)
    #y_points   = np.array(y_points)
    return end_points, y_points

In [15]:
plt.rcParams['figure.figsize'] = [16, 10]

In [16]:
# Start timer to measure segmentation time
start = time.time()

SLAP_ROI = np.zeros(segsperplane.shape)
SLAP_Blocks = np.zeros(segsperplane.shape)
roi_ID = 2
DP_Value = 1
for z in range(segsperplane.shape[0]):
    
    
    segs_colors = np.unique(segsperplane[z,:,:]).astype(int)
    
    
    
    for i in segs_colors[1:]:
        plane = segsperplane[z,:,:].copy()
        plane[plane !=i]=0
        plane[plane>0]=1
        points = np.where(plane==1)

        endpoints, junctions = find_skeleton_points(points, plane)


        if len(junctions) > 0:
            #print("this many y-junctions", len(junctions))
            for junc in junctions:
                plane[junc[1], junc[0]] = 0
            line_fragments = measure.label(plane)

            cut_lines = np.unique(line_fragments).astype(int)
            for line in cut_lines[1:]:
                subplane = line_fragments.copy()

                subplane[subplane !=line]=0
                subplane[subplane>0]=1

                points = np.where(subplane==1)

                endpoints, junctions = find_skeleton_points(points, plane)

                if len(endpoints)>1:
                    #print(endpoints)
                    points  = list(zip(points[1], points[0]))
                    points =  DouglasPeucker(points, 1)
                    pointArray = np.array(points)
                    sortedPoints = np.zeros_like(pointArray)
                    kdTree = KDTree(pointArray)
                    for i, c in enumerate(kdTree.query([0,0], k=pointArray.shape[0])[1]):
                        sortedPoints[i, : ] = pointArray[c, :]

                    for v in range(sortedPoints.shape[0]-1):
                        block, _ =  segmentGen(sortedPoints[v,:], sortedPoints[v+1,:], dendrites[z, :, :])
                        flipped =block
                        oneSeg = flipped*dendrites[z, :, :]
                        oneSeg = dilation(oneSeg)
                        flipped[flipped >0]= roi_ID
                        oneSeg[oneSeg >0]= roi_ID
                        roi_ID += 1
                        SLAP_Blocks[z, :, :] = SLAP_Blocks[z, :, :] + flipped
                        SLAP_ROI[z, :, :] = SLAP_ROI[z, :, :] + oneSeg





        if len(endpoints)>1:
            points = np.where(plane==1)
            points  = list(zip(points[1], points[0]))
            points =  DouglasPeucker(points, 1)
            pointArray = np.array(points)
            sortedPoints = np.zeros_like(pointArray)
            kdTree = KDTree(pointArray)
            for i, c in enumerate(kdTree.query([0,0], k=pointArray.shape[0])[1]):
                sortedPoints[i, : ] = pointArray[c, :]

            for v in range(sortedPoints.shape[0]-1):
                block, _ =  segmentGen(sortedPoints[v,:], sortedPoints[v+1,:], dendrites[z, :, :])
                flipped =block
                oneSeg = flipped*dendrites[z, :, :]
                oneSeg = dilation(oneSeg)
                flipped[flipped >0]= roi_ID
                oneSeg[oneSeg >0]= roi_ID
                roi_ID += 1
                SLAP_Blocks[z, :, :] = SLAP_Blocks[z, :, :] + flipped
                SLAP_ROI[z, :, :] = SLAP_ROI[z, :, :] + oneSeg

soma = labels[0,:,:,:].copy()
soma[labels[0,:,:,:]!=1]=0
soma = np.array(soma, bool)
soma = remove_small_objects(soma,250, connectivity=2)
SLAP_ROI[soma==1]=1                    
SLAP_ROI = SLAP_ROI.astype(int)   
SLAP_Blocks = SLAP_Blocks.astype(int)


end = time.time()


In [17]:
print('Time Elapsed: ', end - start)
neuron_image = image

Time Elapsed:  10.458990812301636


In [27]:
test = napari.Viewer()
test.add_image(neuron_image, name='Neuron', scale=(5, 1, 1), colormap='gray', blending='additive')
#test.add_labels(segsperplane, name='Segments', scale=(5, 1, 1), blending='additive')
test.add_labels(SLAP_ROI, name='ROI', scale=(5, 1, 1), color=labelColorDict, blending='additive')
#test.add_labels(SLAP_Blocks, name='Blocks', scale=(5, 1, 1), blending='additive')

<Labels layer 'ROI' at 0x7fc567f6fd00>

In [26]:

labelColorDict = {}
labelColorDict[0] = (0,0,0)
_max = np.max(SLAP_ROI)
for labelInt in np.unique(SLAP_ROI):
    cmap = cm.Spectral(labelInt/_max)
    labelColorDict[labelInt] = (cmap[0],cmap[1],cmap[2])


In [ ]:
pip install numpy==1.23.5

In [ ]:
test.add_labels(dendrites, name='Blocks', scale=(5, 1, 1), blending='additive')

In [ ]:
points = np.zeros((len(np.unique(SLAP_ROI)), 3))
for index, labelID in enumerate(np.unique(SLAP_ROI)):
    com_measure = np.zeros_like(SLAP_ROI)
    com_measure[SLAP_ROI==labelID] = 1
    points[index, :] = center_of_mass(com_measure)
    

In [ ]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=3, algorithm='brute').fit(points)
distances, indices = nbrs.kneighbors(points)
indices


In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.gca(projection='3d')

for index in range(indices.shape[0]):
    z = [points[indices[index, 0], 0], points[indices[index, 1], 0]]
    x = [points[indices[index, 0], 1], points[indices[index, 1], 1]]
    y = [points[indices[index, 0], 2], points[indices[index, 1], 2]]
    ax.plot(x, y, z, label='parametric curve')
    
    z = [points[indices[index, 0], 0], points[indices[index, 2], 0]]
    x = [points[indices[index, 0], 1], points[indices[index, 2], 1]]
    y = [points[indices[index, 0], 2], points[indices[index, 2], 2]]
    ax.plot(x, y, z, label='parametric curve')
    

    
    
plt.show()